# Extracting the Candidates from the Markdown files

The next step is to get the candidates from the Markdown files. The assumption is that PymuPDF4LLM has extracted all candidates properly in Markdown tables. With this, we will remove all non-Markdown table data and process them based on header-matching, and sort them by the order.

In [12]:
import fs from "node:fs"
import path from "node:path"

In [13]:
let data = fs.readFileSync('./OV.md', 'utf8');

In [14]:
function sortCandidate (a: string, b: string) {
  const firstNumber = Number(a.match(/^(\d+)\.?/)[1])
  const secondNumber = Number(b.match(/^(\d+)\.?/)[1])
  return firstNumber - secondNumber
}

In [15]:
function extractCandidates (data: string) {
  let list = {}
  let currentPosition = ""
  
  // Go through the file line-by line
  
  for (const line of data.split('\n')) {
    // const line = (data as string).split('\n')[91]
    const tableMatcherRegex = /\|((\\\|)|[^\r\n\|])+/g                   // Capture cells starting with the pipe character, except if the pipe character is escaped
    let row = line.match(tableMatcherRegex)
    
    if (!row || row.length === 0) continue                               // If there are no matches, this is not a table, skip the line
    
    row = row
    .map(match => match.replace(/^\|/, "").trim())                     // Remove the starting pipe character, and remove extra whitespaces
    .filter(match => match)                                            // And remove empty strings
    
    if (row[0].replaceAll('-', '') === '') continue                      // If the first cell is a header separator (multiple dashes), skip the line
    
    const isHeader = /^[^\d].*Vote for (\d+)/.exec(row[0])               // If the first cell does not start with a number, it's probably a header
    
    if (isHeader) {
      const position = row[0].split("/")[0].trim()                       // Get the position
      const voteFor = Number(isHeader[1])                                        // Count the "vote for"
      
      if (currentPosition !== position) {                                // If the position is different from the current position being worked on,
        currentPosition = position
        list[position] = {                                               // Write a blank position property
          candidates: [],
          header: row[0],
          voteFor
        }
      }
      continue                                                            // And skip the rest
    } else {                                                              // Else, append the candidates to the row.
      list[currentPosition].candidates =
      list[currentPosition].candidates.concat(row)
    }
  }
  
  // When the list is complete, sort and count the candidates
  
  for (const position of Object.keys(list)) {
    list[position].candidates = list[position].candidates.sort(sortCandidate)
    list[position].count = list[position].candidates.length
  }

  return list
}


In [16]:
let national = extractCandidates(data)
await Deno.jupyter.display(
  { 'text/json': JSON.stringify(national, null, 2) },
  { raw: true }
)


Nice. We finished the basic extraction of candidates to a list.

The partylist count does not match as the [Wage Hike partylist withdrew their candidacy](https://www.inquirer.net/423441/party-list-group-wage-hike-withdraws-bid-from-2025-polls/) last December 17, 2024.

We now write the results down into `.json` files.

In [84]:
// Skippable. Writes a file. 
fs.writeFileSync('./national.json', JSON.stringify(national))

Let's test the code to include local candidates and add some sanity checks with it.

In [17]:
// Skippable. Test.
let akbarRaw = fs.readFileSync('./md/BARMM.BASILAN.AKBAR.md', 'utf8')


In [18]:
function extractCandidatesWithSanityChecks (data: string) {
  let nationalPosts = Object.keys(national)
  let list = extractCandidates(data);

  if (nationalPosts.some(nationalPost => list[nationalPost].count !== national[nationalPost].count)) {
    throw new Error(`${nationalPost} count mismatch`)
  }
  
  for (const nationalPost of nationalPosts) {
    delete list[nationalPost]
  }

  return list
}

In [99]:
// Skippable. Test.
let akbar = extractCandidatesWithSanityChecks(akbarRaw)
await Deno.jupyter.display(
  { 'text/json': JSON.stringify(akbar, null, 2) },
  { raw: true }
)


Time to run it for all files.

In [19]:
const markdownFiles = fs.readdirSync('./md')

In [11]:
// Skippable. Test.
let prevPercent = 0.0

for (const [index, markdownFile] of markdownFiles.entries()) {
  let markdownString = fs.readFileSync(`./md/${markdownFile}`, 'utf8')
  let cityMuniName = path.parse(markdownFile).name
  let cityMuni = ''

  try {
    cityMuni = extractCandidatesWithSanityChecks(markdownString)
  } catch (e) {
    console.error('Error at: ', cityMuniName, e)
  }

  if (cityMuni === '') throw new Error('Failed to read ', cityMuniName)

  fs.writeFileSync(`./json/${cityMuniName}.json`, JSON.stringify(cityMuni))

  let percent = ((index + 1) / markdownFiles.length) * 100.0
  if (Math.floor(prevPercent / 10.0) < Math.floor(percent / 10.0)) {
    console.log(`${Math.round(percent)}% complete`)
    prevPercent = percent
  }
}


Error at:  BARMM.BASILAN.AKBAR ReferenceError: extractCandidatesWithSanityChecks is not defined
    at <anonymous>:7:5


Error: Failed to read 

## Aggregate Stats 

With everything parsed, we now get the total number of candidates, the positions to elect them in, and if there are any discrepancies.

In [20]:
// import { display } from 'https://deno.land/x/display/mod.ts'
import * as pl from 'npm:nodejs-polars'


In [21]:
const jsonFiles = fs.readdirSync('./json')

In [22]:
let prevPercent = 0.0

let candidateTally = {}

for (const position of Object.keys(national)) {
  candidateTally[position] = {
    voteFor: national[position].voteFor || 0,
    count: national[position].count || 0,
  }
}

for (const [index, jsonFile] of jsonFiles.entries()) {
  let jsonString = fs.readFileSync(`./json/${jsonFile}`, 'utf8')
  // let cityMuniName = path.parse(jsonFile).name
  let cityMuni = JSON.parse(jsonString)

  for (const position of Object.keys(cityMuni)) {
    candidateTally[position] = {
      voteFor:
        (cityMuni[position]?.voteFor ?? 0),
      count: (candidateTally[position]?.count ?? 0) + cityMuni[position].count ?? 0,
    }
  }

  let percent = ((index + 1) / markdownFiles.length) * 100.0
  if (Math.floor(prevPercent / 10.0) < Math.floor(percent / 10.0)) {
    console.log(`${Math.round(percent)}% complete`)
    prevPercent = percent
  }
}

let dataFrameObj = {
  position: [],
  voteFor: [],
  count: [],
}

for (const position of Object.keys(candidateTally)) {
  dataFrameObj.position.push(position)
  dataFrameObj.voteFor.push(candidateTally[position].voteFor)
  dataFrameObj.count.push(candidateTally[position].count)
}

let dataFrame = new pl.DataFrame(dataFrameObj)

dataFrame
// await display(dataFrame)

10% complete
20% complete
30% complete
40% complete
50% complete
60% complete
70% complete
80% complete
90% complete
100% complete


position,voteFor,count
SENATOR,12,66
PARTY LIST,1,155
"MEMBER, HOUSE OF REPRESENTATIVES",1,3931
PROVINCIAL GOVERNOR,1,4571
PROVINCIAL VICE-GOVERNOR,1,3903
"MEMBER, SANGGUNIANG PANLALAWIGAN",5,12336
MAYOR,1,3951
VICE-MAYOR,1,3622
"MEMBER, SANGGUNIANG BAYAN",8,27427
BARMM PARTY REPRESENTATIVES,1,763


## Generating the local mapping file

To include candidates for local posts, we have the user search their province and city/municipality they are registered in. Sometimes, cities are further divided into districts.

From the filename of the Markdown files, we will now generate a CSV file containing the following:
- Identifier: The Markdown filename (format: `<region>.<province>.<cityMunicipality>` or `NCR.<cityMunicipality>`)
- Province: The name of the province (or NCR)
- City (and District) or Municipality: The name of the city (and district) or municipality

We then will use the Philippine Standard Geographic Code (PSGC) of 3Q 2024 ([source](https://psa.gov.ph/classification/psgc), up to date as of September 30, 2024) to align proper spelling of the provinces and cities/municipalities through Levenshtein distance.

In [23]:
let dataFrameMappingObj = {
  identifier: [],
  province: [],
  cityMunicipality: []
}

for (const markdownFile of markdownFiles) {
  let identifier = path.parse(markdownFile).name
  let splits = identifier.split('.')

  if (splits.length === 2) {
    dataFrameMappingObj.identifier.push(identifier)
    dataFrameMappingObj.province.push(splits[0])
    dataFrameMappingObj.cityMunicipality.push(splits[1])
  }

  if (splits.length === 3) {
    dataFrameMappingObj.identifier.push(identifier)
    dataFrameMappingObj.province.push(splits[1])                          // Skip the region
    dataFrameMappingObj.cityMunicipality.push(splits[2])
  }
}

1681

In [24]:
let dataFrameMapping = new pl.DataFrame(dataFrameMappingObj)
  .sort('province')
  .select(
    'identifier',
    pl.col('province').str.replaceAll('_', ' '),
    pl.col('cityMunicipality').str.replaceAll('_', ' ')
  )
  
dataFrameMapping


identifier,province,cityMunicipality
BFT_CAR.ABRA.BANGUED,ABRA,BANGUED
BFT_CAR.ABRA.BOLINEY,ABRA,BOLINEY
BFT_CAR.ABRA.BUCAY,ABRA,BUCAY
BFT_CAR.ABRA.BUCLOC,ABRA,BUCLOC
BFT_CAR.ABRA.DAGUIOMAN,ABRA,DAGUIOMAN
BFT_CAR.ABRA.DANGLAS,ABRA,DANGLAS
BFT_CAR.ABRA.DOLORES,ABRA,DOLORES
BFT_CAR.ABRA.LACUB,ABRA,LACUB
BFT_CAR.ABRA.LAGANGILANG,ABRA,LAGANGILANG
BFT_CAR.ABRA.LAGAYAN,ABRA,LAGAYAN


In [25]:
let dataFrameProvinces = dataFrameMapping
  .select(pl.col('province').unique().sort().str.replaceAll('_', ' '))
  .toSeries()
  .toArray()
  
dataFrameProvinces

[
  "ABRA",              "AGUSAN DEL NORTE",   "AGUSAN DEL SUR",
  "AKLAN",             "ALBAY",              "ANTIQUE",
  "APAYAO",            "AURORA",             "BASILAN",
  "BATAAN",            "BATANES",            "BATANGAS",
  "BENGUET",           "BILIRAN",            "BOHOL",
  "BUKIDNON",          "BULACAN",            "CAGAYAN",
  "CAMARINES NORTE",   "CAMARINES SUR",      "CAMIGUIN",
  "CAPIZ",             "CATANDUANES",        "CAVITE",
  "CEBU",              "COTABATO",           "DAVAODELNORTE",
  "DAVAODELSUR",       "DAVAODEORO",         "DAVAOOCCIDENTAL",
  "DAVAOORIENTAL",     "DINAGAT ISLANDS",    "EASTERN SAMAR",
  "GUIMARAS",          "IFUGAO",             "ILOCOS SUR",
  "ILOCOSNORTE",       "ILOILO",             "ISABELA",
  "KALINGA",           "LAGUNA",             "LANAODELSUR",
  "LAUNION",           "LDN",                "LEYTE",
  "MAGUINDANAODELSUR", "MAGUINDANAONORTE",   "MARINDUQUE",
  "MASBATE",           "MISOR",              "MISSOCC",
  "MPROVINCE

In [26]:
// Here's a list of all provinces from the PSGC, with NCR and SGA added at the end.
let PSGCprovinceswithcoding = pl.readCSV('./psgc/provinces.csv', {
  dtypes: { province: pl.DataType.String, code: pl.DataType.String },
})

let PSGCprovinces = PSGCprovinceswithcoding["province"].toArray()

In [27]:
`${PSGCprovinces.length} ${dataFrameProvinces.length}`

"84 84"

In [75]:
import { distance, closest } from 'https://deno.land/x/fastest_levenshtein/mod.ts'

In [29]:
let provinceCorrections = {
  "LDN": "LANAO DEL NORTE",
  "MISOR": "MISAMIS ORIENTAL",
  "MISSOCC": "MISAMIS OCCIDENTAL",
  "MPROVINCE": "MOUNTAIN PROVINCE",
  "NCR": "NATIONAL CAPITAL REGION (NCR) - METRO MANILA",
  "SGA": "SPECIAL GEOGRAPHIC AREA",
}

let dataFrameBetterProvinces = new pl.DataFrame({
  old: dataFrameProvinces,
  new: dataFrameProvinces.map((province) => {
    if (provinceCorrections[province]) {
      return provinceCorrections[province]
    } else {
      return closest(
        province,
        pl.Series(PSGCprovinces).str.toUpperCase().toArray()
      )
    }
  }),
  score: dataFrameProvinces.map((province) => {
    if (provinceCorrections[province]) {
      return -1
    } else {
      return distance(province, closest(
        province,
        pl.Series(PSGCprovinces).str.toUpperCase().toArray()
      ))
    }
  }),
})

await Deno.jupyter.display({"text/csv": dataFrameBetterProvinces.sort("score", true).writeCSV().toString()}, {raw: "true"})

In [30]:
dataFrameBetterProvinces
  .select(pl.col('new').unique().sort())
  .toSeries()
  .toArray().length

84

In [31]:
// Here's a list of all cities and municipalities with their corresponding code. I have added old names to the end of the list with the same code as their new ones so that there are more chances to match.
let PSGCcitymuniwithcoding = pl.readCSV('./psgc/citiesMunicipalities.csv', {
  dtypes: { cityMunicipality: pl.DataType.String, code: pl.DataType.String },
})

In [32]:
// We aggregate the original table with the better province and the province code.
let joinBetterProvince = dataFrameMapping
  .join(
    dataFrameBetterProvinces.select(
      pl.col('old').alias('province'),
      pl.col('new').alias('newProvince')
    ),
    { on: 'province' }
  )
  .join(
    PSGCprovinceswithcoding.select(
      pl.col('province').alias('newProvince').str.toUpperCase(), 
      pl.col('code').alias('provinceCode')
    ), 
    { on: 'newProvince'}
  )

  joinBetterProvince

identifier,province,cityMunicipality,newProvince,provinceCode
BFT_CAR.ABRA.BANGUED,ABRA,BANGUED,ABRA,14001
BFT_CAR.ABRA.BOLINEY,ABRA,BOLINEY,ABRA,14001
BFT_CAR.ABRA.BUCAY,ABRA,BUCAY,ABRA,14001
BFT_CAR.ABRA.BUCLOC,ABRA,BUCLOC,ABRA,14001
BFT_CAR.ABRA.DAGUIOMAN,ABRA,DAGUIOMAN,ABRA,14001
BFT_CAR.ABRA.DANGLAS,ABRA,DANGLAS,ABRA,14001
BFT_CAR.ABRA.DOLORES,ABRA,DOLORES,ABRA,14001
BFT_CAR.ABRA.LACUB,ABRA,LACUB,ABRA,14001
BFT_CAR.ABRA.LAGANGILANG,ABRA,LAGANGILANG,ABRA,14001
BFT_CAR.ABRA.LAGAYAN,ABRA,LAGAYAN,ABRA,14001


In [100]:
let cityMuniCorrections = {
  'SAMALISLAND KAPUTIAN DIST':
    'ISLAND GARDEN CITY OF SAMAL (KAPUTIAN DISTRICT)',
  'SAMALISLAND BABAK DIST': 'ISLAND GARDEN CITY OF SAMAL (BABAK DISTRICT)',
  'SAMALISLAND SAMAL DIST': 'ISLAND GARDEN CITY OF SAMAL (SAMAL DISTRICT)',
  'CITY OF MABALACAT': 'CITY OF MABALACAT',
  'CDOCITY 1ST': 'CITY OF CAGAYAN DE ORO (1ST DISTRICT)',
  'CDOCITY 2ND': 'CITY OF CAGAYAN DE ORO (2ND DISTRICT)',
  'CITY OF SAN JOSE': 'SAN JOSE CITY',
}

// Fancy function that matches cities or municipalities with their proper name from the PSGC.
// 1. It also handles formatting of districts
// 2. Checks two names (Angeles City and City of Angeles)
// 3. Checks names by province code or if it's a city, because some cities have weird PSGC codes

function getNewCityMuni(cityMuni: string, index: number) {
  if (cityMuniCorrections[cityMuni]) {
    // Manually correct the name of the city or municipality.
    return [cityMuniCorrections[cityMuni], -1]
  } else {
    // City or municipality name cleanup
    let cleanCityMuni = cityMuni
    let altCityMuni = ''
    let district = ''

    // Get the district and remove it from the city or municipality name
    let districtMatcher = /(.*)(\d\w{2}|(?: )\w+)\s?DIST(?:RICT)?/g
    let districtMatch = [...cleanCityMuni.matchAll(districtMatcher)]
      .flat()
      .filter((x) => x)

    if (districtMatch.length === 3) {
      district = `(${districtMatch[2].trim()} DISTRICT)`
      cleanCityMuni = districtMatch[1].trim()
    }

    // Get an alternate name for cities (e.g. Davao City becomes City of Davao)
    let cityMatcher = /(.*)(?=CITY)/g
    let cityMatch = [...cleanCityMuni.matchAll(cityMatcher)]
      .flat()
      .filter((x) => x)

    if (cityMatch.length === 2) {
      altCityMuni = `CITY OF ${cityMatch[1].trim()}`
    }

    // Get the distance and possible lookup list.
    let lookupList = PSGCcitymuniwithcoding.filter(
      pl
        .col('code')
        .str.contains(
          RegExp(`^${joinBetterProvince['provinceCode'].get(index)}`)
        )
        .or(pl
          .col('cityMunicipality')
          .str.contains(
            'City')
          )
    )
      ['cityMunicipality'].str.toUpperCase()
      .toArray()

    // If there's an alternate city name, compare the two and get the option with the lowest score. If not, return the closest match and the score.
    if (altCityMuni !== '') {
      let option1 = closest(cleanCityMuni, lookupList)
      let option2 = closest(altCityMuni, lookupList)
      let score1 = distance(cleanCityMuni, option1)
      let score2 = distance(altCityMuni, option2)

      if (score1 < score2) {
        return [`${option1}${district !== '' ? ' ' + district : ''}`, score1]
      } else {
        return [`${option2}${district !== '' ? ' ' + district : ''}`, score2]
      }
    } else {
      let newCityMuni = closest(cleanCityMuni, lookupList)
      let score = distance(cleanCityMuni, newCityMuni)

      return [`${newCityMuni}${district !== '' ? ' ' + district : ''}`, score]
    }
  }
}

let betterProvinceCityMuni = joinBetterProvince
  .select('identifier', 'newProvince', 'provinceCode', 'cityMunicipality')
  .join(
    new pl.DataFrame({
      identifier: joinBetterProvince['identifier'],
      newCityMunicipality: joinBetterProvince['cityMunicipality']
        .toArray()
        .map((cityMuni: string, index: number) => {
          const [newCityMuni] = getNewCityMuni(cityMuni, index)
          return newCityMuni
        }),
      score: joinBetterProvince['cityMunicipality']
        .toArray()
        .map((cityMuni: string, index: number) => {
          const [, score] = getNewCityMuni(cityMuni, index)
          return score
        }),
    }),
    { on: 'identifier' }
  )
  .sort('score', true)


In [101]:
betterProvinceCityMuni

identifier,newProvince,provinceCode,cityMunicipality,newCityMunicipality,score
BARMM.MAGUINDANAODELSUR.GENSKPENDATUN,MAGUINDANAO DEL SUR,19088,GENSKPENDATUN,GEN. S.K. PENDATUN,5
BFT_R9.ZAMBOANGADELNORTE.PRESMANUELAROXAS,ZAMBOANGA DEL NORTE,09072,PRESMANUELAROXAS,PRES. MANUEL A. ROXAS,5
BFT_R9.ZAMBOANGADELNORTE.SERGIOOSMEnASR,ZAMBOANGA DEL NORTE,09072,SERGIOOSMEnASR,SERGIO OSMEÑA SR.,4
R11.DAVAODELNORTE.BRAULIOEDUJALI,DAVAO DEL NORTE,11023,BRAULIOEDUJALI,BRAULIO E. DUJALI,3
BARMM.LANAODELSUR.TAGOLOAN,LANAO DEL SUR,19036,TAGOLOAN,TAGOLOAN II,3
BFT_R9.ZAMBOANGADELNORTE.LEONTPOSTIGO,ZAMBOANGA DEL NORTE,09072,LEONTPOSTIGO,LEON T. POSTIGO,3
BFT_R9.ZAMBOANGADELSUR.VINCENZOASAGUN,ZAMBOANGA DEL SUR,09073,VINCENZOASAGUN,VINCENZO A. SAGUN,3
BARMM.BASILAN.HADJI_MUTHAMAD,BASILAN,19007,HADJI MUTHAMAD,HADJI MUHTAMAD,2
BFT_R2.CAGAYAN.SANTONInO,CAGAYAN,02015,SANTONInO,SANTO NIÑO,2
BFT_R7.CEBU.SANREMIGO,CEBU,07022,SANREMIGO,SAN REMIGIO,2


In [105]:
await Deno.jupyter.display(
  {
    'text/csv': betterProvinceCityMuni
      .sort('score', true)
      .writeCSV()
      .toString(),
  },
  { raw: 'true' }
)


In [113]:
// For backwards-compatibility of localMapping, we include the ID and the link here:

let localMapping = betterProvinceCityMuni
  .sort(['newProvince', 'newCityMunicipality'])
  .select(
    pl.col('identifier'),
    pl.col('newProvince').alias('province'),
    pl.col('newCityMunicipality').alias('cityMunicipality')
  )
  .join(
    new pl.DataFrame({
      identifier: joinBetterProvince['identifier'],
      id: joinBetterProvince['identifier'].toArray().map((_: string, index: number) => index + 1),
      link: joinBetterProvince['identifier']
        .toArray()
        .map(
          (identifier: string) =>
            `https://comelec.gov.ph/php-tpls-attachments/2025NLE/2025BallotFace/${identifier.replaceAll(".", "/")}.pdf`
        ),
    }),
    { on: "identifier"}
  )

In [114]:
// Skippable. Writes a file.
fs.writeFileSync('./localMapping.json', JSON.stringify(localMapping.toRecords()))